In [1]:
!nvidia-smi

Thu May 14 03:46:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  N/A |
| 26%   28C    P8     7W / 180W |      1MiB / 16278MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!ls

Phenotypic_V1_0b_preprocessed1.csv  correlations_matrixcc400.pkl
Untitled.html			    data
asd_classify.html		    datasets
asd_classify.ipynb		    download_abide.py
asd_classify_conv_mod.html	    storage
asd_classify_conv_mod.ipynb


In [3]:
# !python download_abide.py

In [2]:
p_ROI = "cc400"
p_fold = 10
#p_center = "Stanford"
#p_mode = "whole"
#p_augmentation = True
#p_Method = "ASD-DiagNet"

In [3]:
parameter_list = [p_ROI]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
# print("per Center or whole: ",p_mode)
# if p_mode == 'percenter':
#     print("Center's name: ",p_center)
# print("Method's name: ",p_Method)
# if p_Method == "ASD-DiagNet":
#     print("Augmentation: ",p_augmentation)

*****List of patameters****
ROI atlas:  cc400


In [13]:
!pip install pyprind 

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [5]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [22]:
# import glob

In [6]:
data_main_path = './data/rois_cc400/' #cc400#path to time series data
flist = os.listdir(data_main_path)
# flist = glob.glob('./data/rois_cc400/*.1D')
print(len(flist))

1035


In [7]:
for f in range(len(flist)):
    flist[f] = get_key(flist[f])
    

df_labels = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv')#path 

df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})
print(len(df_labels))

labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label

1112


In [8]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename] 


def get_corr_data(filename):
    # print(filename)
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
            # print(df)
            
    with np.errstate(invalid="ignore"):
        # print(df.T)
        # print(np.corrcoef(df.T).shape)
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        # print(mask)
        m = ma.masked_where(mask == 1, mask)
        # print(m)

        return ma.masked_where(m, corr).compressed()

def get_corr_matrix(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    return regions


In [9]:
if not os.path.exists('./correlations_matrix'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr_mat = {}
    for f in flist:
      
        lab = get_label(f)
        all_corr_mat[f] = (get_corr_matrix(f), lab)
        pbar.update()

    print('Corr-computations finished')

    pickle.dump(all_corr_mat, open('./correlations_matrix'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')
else: 
    all_corr_mat = pickle.load(open('./correlations_matrix'+p_ROI+'.pkl', 'rb'))
    print('loaded')

loaded


In [10]:
all_corr_mat['Pitt_0050003'][0].shape

(392, 392)

In [1]:
# sorted(flist)

In [14]:
# len(eig_data[flist[0]]['eigvals'])

In [2]:
# sorted(labels)

In [11]:
class CC400Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None):
        if pkl_filename is not None:           
            print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
            
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Eigther PKL file or data is needed!')
            return 

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')
              
        self.num_data = len(self.flist)
        
    def __getitem__(self, index):
        
        fname = self.flist[index]
        data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode) 
        
#         ## only taking upper triangle.. rest to zero
#         data = np.array(data)
#         data = np.triu(data,1)
        
        data = torch.FloatTensor(data)
        data = torch.unsqueeze(data,0)


#       print(data.shape)

        #print(s.shape)
        label = (self.labels[index],)
        # print(label)
        
        return data, torch.FloatTensor(label)            

    def __len__(self):
        return self.num_data

In [12]:
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=32, 
               num_workers=1, mode='train'):
    
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False

    dataset = CC400Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list)

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)
    return data_loader

In [25]:
!pip install torchsummary

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [16]:
from torchsummary import summary

Evovle Norm Act

In [13]:
# evolving normalisation activation layers  

from torch.nn import init
from torch.nn.parameter import Parameter

def instance_std(x, eps=1e-5):
    N,C,H,W = x.size()
    x1 = x.reshape(N*C,-1)
    var = x1.var(dim=-1, keepdim=True)+eps
    return var.sqrt().reshape(N,C,1,1)

def group_std(x, groups, eps = 1e-5):
    N, C, H, W = x.size()
    x1 = x.reshape(N,groups,-1)
    var = (x1.var(dim=-1, keepdim = True)+eps).reshape(N,groups,-1)
    return (x1 / var.sqrt()).reshape(N,C,H,W)


class BatchNorm2dRelu(nn.Module):
    def __init__(self,in_channels):
        super(BatchNorm2dRelu,self).__init__()
        self.layer = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True))
    def forward(self, x):
        output = self.layer(x)
        return output


class EvoNorm2dB0(nn.Module):
    def __init__(self,in_channels,nonlinear=True,momentum=0.9,eps = 1e-5):
        super(EvoNorm2dB0, self).__init__()
        self.nonlinear = nonlinear
        self.momentum = momentum
        self.eps = eps
        self.gamma = Parameter(torch.Tensor(1,in_channels,1,1))
        self.beta = Parameter(torch.Tensor(1,in_channels,1,1))
        if nonlinear:
            self.v = Parameter(torch.Tensor(1,in_channels,1,1))
        self.register_buffer('running_var', torch.ones(1, in_channels, 1, 1))
        self.reset_parameters()

    def reset_parameters(self):
        init.ones_(self.gamma)
        init.zeros_(self.beta)
        if self.nonlinear:
            init.ones_(self.v)

    def forward(self, x):
        N, C, H, W = x.size()
        if self.training:
            x1 = x.permute(1, 0, 2, 3).reshape(C, -1)
            var = x1.var(dim=1).reshape(1, C, 1, 1)
            self.running_var.copy_(self.momentum * self.running_var + (1 - self.momentum) * var)
        else:
            var = self.running_var
        if self.nonlinear:
            den = torch.max((var+self.eps).sqrt(), self.v * x + instance_std(x))
            return x / den * self.gamma + self.beta
        else:
            return x * self.gamma + self.beta


class EvoNorm2dS0(nn.Module):
    def __init__(self,in_channels,groups=8,nonlinear=True):
        super(EvoNorm2dS0, self).__init__()
        self.nonlinear = nonlinear
        self.groups = groups
        self.gamma = Parameter(torch.Tensor(1,in_channels,1,1))
        self.beta = Parameter(torch.Tensor(1,in_channels,1,1))
        if nonlinear:
            self.v = Parameter(torch.Tensor(1,in_channels,1,1))
        self.reset_parameters()

    def reset_parameters(self):
        init.ones_(self.gamma)
        init.zeros_(self.beta)
        if self.nonlinear:
            init.ones_(self.v)
    def forward(self, x):
        if self.nonlinear:
            num = torch.sigmoid(self.v * x)
            std = group_std(x,self.groups)
            return num * std * self.gamma + self.beta
        else:
            return x * self.gamma + self.beta

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
from torchsummary import summary

In [23]:
class convnet(nn.Module):
    def __init__(self):
        super(convnet,self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, (1,392))  ## with 32 channels 0.788 acc  ... exp 
        self.conv2 = nn.Conv2d(32, 64, (392,1))
        self.conv3 = nn.Conv2d(1,32, (392,1))
        self.conv4 = nn.Conv2d(32,64,(1,392) )
        
        self.normact1 = EvoNorm2dS0(32)
        self.normact2 = EvoNorm2dS0(64) 
        
#         self.conv1 = nn.Conv2d(1, 64, (392,1))   ## normal cnn ## not using this one
#         self.conv2 = nn.Conv2d(64, 128, (1,392)) 
        
#         self.conv1 = nn.Conv2d(1, 64, (1,392))   ## normal cnn ## used this one (1x392)
#         self.conv2 = nn.Conv2d(64, 128, (392,1))  ##  Too.. Big .. :\
        

#         self.normact1 = EvoNorm2dS0(64)   ## used this one..
#         self.normact2 = EvoNorm2dS0(128)


#         self.bn1 = nn.BatchNorm2d(64)
#         self.bn2 = nn.BatchNorm2d(128)


        self.fc = nn.Linear(128,1)
#         self.fc = nn.Linear(128,1)

        self.dropout = nn.Dropout(p= 0.6)   ## in 0.6 taken 
    
    def forward(self, x):
        
        
        x1 = self.normact1(self.conv1(x))    ## exp 
        x2 = self.normact2(self.conv2(x1))

        x3 = self.normact1(self.conv3(x))
        x4 = self.normact2(self.conv4(x3))
       
    
        x = torch.cat((x2,x4),1)
        
#         x = self.dropout(self.normact1(self.conv1(x)))  ## 0.817 acc ... give a check to it 
#         x = self.normact2(self.conv2(x)) 
        
#         x = self.normact1(self.conv1(x)) 
#         x = self.normact2(self.conv2(x))

#         x = self.bn1(F.relu(self.conv1(x)))
#         x = self.bn2(F.relu(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        
        x = self.fc(self.dropout(x))  

        return F.sigmoid(x)

m = convnet() 
m.to(device)

summary(m, (1 ,392 , 392 ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 392, 1]          12,576
       EvoNorm2dS0-2           [-1, 32, 392, 1]               0
            Conv2d-3             [-1, 64, 1, 1]         802,880
       EvoNorm2dS0-4             [-1, 64, 1, 1]               0
            Conv2d-5           [-1, 32, 1, 392]          12,576
       EvoNorm2dS0-6           [-1, 32, 1, 392]               0
            Conv2d-7             [-1, 64, 1, 1]         802,880
       EvoNorm2dS0-8             [-1, 64, 1, 1]               0
           Dropout-9                  [-1, 128]               0
           Linear-10                    [-1, 1]             129
Total params: 1,631,041
Trainable params: 1,631,041
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.59
Forward/backward pass size (MB): 0.39
Params size (MB): 6.22
Estimat

In [18]:
def train(model, epoch, train_loader ):
    
    model.train()
    batch_loss, ep_loss = 0.0,0.0
    num = 0
    for i,(batch_x,batch_y) in enumerate(train_loader): 
        
        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        out = model(data)

        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        
        batch_loss +=  loss.item() * len(batch_x) 
        
        num += (len(batch_x))
        
    ep_loss = batch_loss / num 
    print('train_epoch_loss: ', ep_loss) 

    return ep_loss


def test(model, test_loader):
    n_test, correct = 0,0

    all_predss=[] 

    y_true, y_pred = [], []

    with torch.no_grad():
        model.eval()

        for i,(batch_x,batch_y) in enumerate(test_loader, 1):

            data = batch_x.to(device)

            out = model(data)

            proba = out.detach().cpu().numpy()

            preds = np.ones_like(proba, dtype=np.int32)
            preds[proba < 0.5] = 0

            all_predss.extend(preds)

            y_arr = np.array(batch_y, dtype=np.int32)
            
            correct += np.sum(preds == y_arr)
            n_test += len(batch_x)
            
            y_true.extend(y_arr.tolist())
            y_pred.extend(proba.tolist())
        
#         loss_avg = loss_total / n_test  
        acc,sens,spec = confusion(y_true,all_predss)        
        
#         print('test_loss_avg:', loss_avg)
#         print('test_acc:', acc )
#         print('test_sens:', sens )
#         print('test_spef:', spec )
        

    return  acc,sens,spec

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [47]:
# len(all_corr[flist[0]][0])

In [24]:
start =time.time()
batch_size = 32  ## for yielding good results in evolve-norm-act 

num_epochs = 300  
p_fold = 10 


crossval_res_kol=[]
# y_arr = np.array([get_label(f) for f in flist])
flist = np.array(sorted(flist))

kk=0 
acc_avg = []
sens_avg = []
spec_avg = []
#     for rp in range(10):

#         print('whole_iterate:', rp)


kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
np.random.seed(3)
np.random.shuffle(flist)
y_arr = np.array([get_label(f) for f in flist])

for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
    train_samples, test_samples = flist[train_index], flist[test_index]
    
#     print(len(train_samples))
#     print(len(test_samples))


    train_loader=get_loader(data=all_corr_mat, samples_list=train_samples, 
                            batch_size=batch_size, mode='train')

    test_loader=get_loader(data=all_corr_mat, samples_list=test_samples, 
                           batch_size=batch_size, mode='test')

    model = convnet()
    model.to(device)
    criterion = nn.BCELoss() 
    optimizer = optim.Adam(model.parameters())

#     print('k_fold:', kk )
    
    best_acc, j = 0.0, 0

    for epoch in range(1, num_epochs+1):
        print('epoch: ', epoch)

        train_losses = train(model, epoch, train_loader)
        acc, sens, spec = test(model, test_loader)
        
        j = j + 1  
        
        if acc > best_acc:
            best_acc = acc
            j = 0
            
        else:  ## early stopping
            print('patience', j)
#         if j == 100:  # early stopping with patience 100  
#             print('test_best_acc:', best_acc)
#             break
        print('test_acc:', acc)
        print('test_best_acc:', best_acc)
                   
    break
    
#     acc_avg.append(acc)
#     sens_avg.append(sens)
#     spec_avg.append(spec)
    
    
finish= time.time()

# print('Avg_test_acc: ', np.mean(acc_avg))
# print('Avg_test_sens: ', np.mean(sens_avg))
# print('Avg_test_spec: ', np.mean(spec_avg))
print(finish-start)
print('Done !')

epoch:  1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train_epoch_loss:  0.6976231358361168
test_acc: 0.6057692307692307
test_best_acc: 0.6057692307692307
epoch:  2
train_epoch_loss:  0.6265081209475705
patience 1
test_acc: 0.5769230769230769
test_best_acc: 0.6057692307692307
epoch:  3
train_epoch_loss:  0.5409867251341889
patience 2
test_acc: 0.5480769230769231
test_best_acc: 0.6057692307692307
epoch:  4
train_epoch_loss:  0.43157814307064296
patience 3
test_acc: 0.5480769230769231
test_best_acc: 0.6057692307692307
epoch:  5
train_epoch_loss:  0.3490129593363638
patience 4
test_acc: 0.5961538461538461
test_best_acc: 0.6057692307692307
epoch:  6
train_epoch_loss:  0.24324563744244848
test_acc: 0.625
test_best_acc: 0.625
epoch:  7
train_epoch_loss:  0.22167643133218767
patience 1
test_acc: 0.5961538461538461
test_best_acc: 0.625
epoch:  8
train_epoch_loss:  0.08943496408415148
patience 2
test_acc: 0.5480769230769231
test_best_acc: 0.625
epoch:  9
train_epoch_loss:  0.06791594856799801
patience 3
test_acc: 0.625
test_best_acc: 0.625
epoch: 